In [3]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# --- 1. Cargar dataset ---
with open("dataset_julio.json", "r", encoding="utf-8") as f:
    data = json.load(f)

pares = data["pares"]
tickets = data["tickets"]

# --- 2. Crear vocabulario ---
productos = list({p for ticket in tickets for p in ticket})
productos.sort()
token2id = {p: i for i, p in enumerate(productos)}
id2token = {i: p for p, i in token2id.items()}
vocab_size = len(productos)

# --- 3. Dataset PyTorch ---
class ProductosDataset(Dataset):
    def __init__(self, pares, token2id):
        self.inputs = []
        self.targets = []
        for par in pares:
            entrada_ids = [token2id[p] for p in sorted(par["entrada"])]
            self.inputs.append(torch.tensor(entrada_ids, dtype=torch.long))
            self.targets.append(token2id[par["siguiente"]])
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

# --- 4. Collate function para padding ---
def collate_fn(batch):
    inputs, targets = zip(*batch)
    lengths = [len(seq) for seq in inputs]
    max_len = max(lengths)
    padded_inputs = torch.zeros(len(inputs), max_len, dtype=torch.long)
    for i, seq in enumerate(inputs):
        padded_inputs[i, :lengths[i]] = seq
    targets = torch.tensor(targets, dtype=torch.long)
    return padded_inputs, targets

dataset = ProductosDataset(pares, token2id)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [4]:
# --- 5. Definir transformer pequeño ---
class MiniTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=4, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # x: [batch, seq_len] -> transformer espera [seq_len, batch, d_model]
        x = self.embedding(x).permute(1,0,2)
        x = self.transformer(x)
        # predecimos solo el último token de la secuencia
        x = self.fc(x[-1])
        return F.log_softmax(x, dim=-1)

model = MiniTransformer(vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

/home/eponlop/apr/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [21]:
# --- 6. Entrenamiento ---
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()
        output = model(batch_inputs)
        loss = criterion(output, batch_targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/50 - Loss: 3.4168
Epoch 2/50 - Loss: 3.1703
Epoch 3/50 - Loss: 3.1286
Epoch 4/50 - Loss: 3.1046
Epoch 5/50 - Loss: 3.0840
Epoch 6/50 - Loss: 3.0719
Epoch 7/50 - Loss: 3.0597
Epoch 8/50 - Loss: 3.0495
Epoch 9/50 - Loss: 3.0428
Epoch 10/50 - Loss: 3.0330
Epoch 11/50 - Loss: 3.0237
Epoch 12/50 - Loss: 3.0177
Epoch 13/50 - Loss: 3.0103
Epoch 14/50 - Loss: 3.0024
Epoch 15/50 - Loss: 2.9955
Epoch 16/50 - Loss: 2.9899
Epoch 17/50 - Loss: 2.9867
Epoch 18/50 - Loss: 2.9828
Epoch 19/50 - Loss: 2.9800
Epoch 20/50 - Loss: 2.9740
Epoch 21/50 - Loss: 2.9697
Epoch 22/50 - Loss: 2.9658
Epoch 23/50 - Loss: 2.9642
Epoch 24/50 - Loss: 2.9612
Epoch 25/50 - Loss: 2.9604
Epoch 26/50 - Loss: 2.9566
Epoch 27/50 - Loss: 2.9547
Epoch 28/50 - Loss: 2.9532
Epoch 29/50 - Loss: 2.9508
Epoch 30/50 - Loss: 2.9486
Epoch 31/50 - Loss: 2.9470
Epoch 32/50 - Loss: 2.9446
Epoch 33/50 - Loss: 2.9442
Epoch 34/50 - Loss: 2.9409


KeyboardInterrupt: 

In [1]:
# --- 7. Ejemplo de predicción ---
def predecir_siguiente(model, secuencia, token2id, id2token):
    model.eval()
    secuencia_ordenada = sorted(secuencia)  # ✅ ordenar entrada
    entrada_ids = torch.tensor([[token2id[p] for p in secuencia_ordenada]], dtype=torch.long)
    with torch.no_grad():
        output = model(entrada_ids)
        probs = output.exp().squeeze()
        # ✅ filtrar productos ya comprados
        for p in secuencia_ordenada:
            probs[token2id[p]] = 0.0
        probs = probs / probs.sum()
        idx = torch.argmax(probs).item()
    return id2token[idx]


ejemplo = ["Cerveza negra", "Chicles", "Cerveza rubia", "Chocolate con leche", "Chocolate negro"]
print("Entrada:", ejemplo)
print("Predicción siguiente producto:", predecir_siguiente(model, ejemplo, token2id, id2token))

Entrada: ['Cerveza negra', 'Chicles', 'Cerveza rubia', 'Chocolate con leche', 'Chocolate negro']


NameError: name 'model' is not defined

In [ ]:
archivo_modelo = "transformer_mercadona_julio.pth"

# --- 1. Guardar modelo ---
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'vocab': token2id  # guardamos el vocabulario también
}, archivo_modelo)

print(f"Modelo guardado en {archivo_modelo}")

In [5]:
archivo_modelo = "transformer_mercadona_julio.pth"

# --- 2. Cargar modelo ---
# Para cargar más tarde:
checkpoint = torch.load(archivo_modelo)

# recrear el modelo y el optimizador con la misma arquitectura
model_cargado = MiniTransformer(vocab_size=len(checkpoint['vocab']))
optimizer_cargado = torch.optim.Adam(model_cargado.parameters(), lr=0.001)

model_cargado.load_state_dict(checkpoint['model_state_dict'])
optimizer_cargado.load_state_dict(checkpoint['optimizer_state_dict'])
token2id = checkpoint['vocab']
id2token = {i: p for p, i in token2id.items()}

model_cargado.eval()  # listo para inferencia
print("Modelo cargado y listo para predecir.")

Modelo cargado y listo para predecir.


/home/eponlop/apr/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [10]:
def predecir_siguiente(model, secuencia, token2id, id2token):
    model.eval()
    secuencia_ordenada = sorted(secuencia)  # ✅ ordenar entrada
    entrada_ids = torch.tensor([[token2id[p] for p in secuencia_ordenada]], dtype=torch.long)
    with torch.no_grad():
        output = model(entrada_ids)
        probs = output.exp().squeeze()
        # ✅ filtrar productos ya comprados
        for p in secuencia_ordenada:
            probs[token2id[p]] = 0.0
        probs = probs / probs.sum()
        idx = torch.argmax(probs).item()
    return id2token[idx]

ejemplo = ["Cerveza negra", "Chicles", "Cerveza rubia", "Chocolate con leche", "Chocolate negro", "Frutos secos"]
print("Entrada:", ejemplo)
prediccion = predecir_siguiente(model_cargado, ejemplo, token2id, id2token)
print("Predicción siguiente producto:", prediccion, type(prediccion))

Entrada: ['Cerveza negra', 'Chicles', 'Cerveza rubia', 'Chocolate con leche', 'Chocolate negro', 'Frutos secos']
Predicción siguiente producto: Galletas dulces <class 'str'>
